In [2]:
# Specify parameters for augmentation

audio_config = {}

audio_config['features_output_dir'] = 'hey_mycroft_validation_mmap'
# audio_config['features_output_dir'] = 'augmented_features_mmap'

audio_config['input_path'] = '/Users/kahrendt/Documents/Hobbies/Machine_Learning/wakeword_tflite/wake-word-benchmark/audio/alexa'
# audio_config['input_path'] = '/Volumes/MachineLearning/audio_samples/recorded_samples/hey_mycroft/dataset_split/positive_samples'
# audio_config['input_path'] = 'generated_samples'
audio_config['input_glob'] = '*.flac'
# audio_config['input_glob'] = '*.wav'
audio_config['impulse_paths'] = None
# audio_config['background_paths'] = None
audio_config['impulse_paths'] = ['mit_rirs']
audio_config['background_paths'] = ['fma']
audio_config['min_clip_duration_s'] = None
audio_config['max_clip_duration_s'] = None
audio_config['max_start_time_from_right_s'] = None
audio_config['augmented_duration_s'] = 3.99

from microwakeword.feature_generation import ClipsHandler

# Load audio clips and prepare them for augmentation

clips_handler = ClipsHandler(
                            input_path=audio_config['input_path'],
                            input_glob=audio_config['input_glob'],
                            impulse_paths=audio_config['impulse_paths'], 
                            background_paths=audio_config['background_paths'], 
                            augmentation_probabilities = {
                                "SevenBandParametricEQ": 0.0,
                                "TanhDistortion": 0.0,
                                "PitchShift": 0.0,
                                "BandStopFilter": 0.0,
                                "AddColorNoise": 0.0,
                                "AddBackgroundNoise": 0.0,
                                "Gain": 0.0,
                                "RIR": 0.0,
                            },
                            augmented_duration_s = audio_config['augmented_duration_s'],
                            max_start_time_from_right_s = audio_config['max_start_time_from_right_s'],
                            max_clip_duration_s = audio_config['max_clip_duration_s'],   
                            min_clip_duration_s = audio_config['min_clip_duration_s'],     
                            remove_silence=False,
)

In [5]:
import IPython

clips_handler.save_random_augmented_clip("augmented_clip.wav")

IPython.display.display(IPython.display.Audio("augmented_clip.wav"))

In [56]:
import microwakeword.inference as inference
from tqdm import tqdm
import IPython
import numpy as np
import os
model = inference.Model('/Users/kahrendt/Documents/Hobbies/Programming/Git-Repositories/microWakeWord/notebooks/trained_models/alexa_mixednet_5_min_jitter/tflite_stream_state_internal_quant/stream_state_internal_quant.tflite')
# model = inference.Model('/Users/kahrendt/Documents/Hobbies/Programming/Git-Repositories/microWakeWord/models/alexa.tflite')
good_counter = 0
bad_counter = 0
all_counter = 0
truncated_clips = clips_handler.clips[:]['audio']
# truncated_clips = tqdm(clips_handler.clips)
# for clip in tqdm(truncated_clips[:100]):
for clip in tqdm(truncated_clips):
# for clip in tqdm(clips_handler.clips):
    # audio_data = clip["audio"]["array"]
    audio_data = clip["array"]
    spectrogram = clips_handler.generate_augmented_spectrogram(audio_data)
    # all_counter +=1
    probabilities = model.predict_spectrogram(spectrogram)#[-103:,:])
    if np.max(probabilities[50:]) < 0.5:
    # if np.max(probabilities[-100:]) < 0.84:
        path_to_clip = clip['path']
        head, tail = os.path.split(path_to_clip)
        # renamed_path_clip = os.path.join(head, 'possibly_bad', tail)
        bad_counter += 1
        # print(tail)
    else:
        good_counter += 1
    
        
    # if not all_counter % 100:
    #     print(good_counter/all_counter)
        
        # os.rename(path_to_clip, renamed_path_clip)
        # print(tail, np.max(probabilities))

        # IPython.display.display(IPython.display.Audio(path_to_clip))
        # IPython.display.display(IPython.display.Audio(renamed_path_clip))
print(good_counter/len(truncated_clips))

100%|██████████| 305/305 [00:01<00:00, 229.75it/s]

0.9901639344262295


In [5]:
from pathlib import Path
import datasets
import microwakeword.inference as inference
import numpy as np

path_to_clips = [str(i) for i in Path('/Volumes/MachineLearning/audio_samples/recorded_samples/hey_mycroft').glob('personal_samples/*.wav')]

audio_dataset = datasets.Dataset.from_dict(
            {"audio": [str(i) for i in path_to_clips]}
        ).cast_column("audio", datasets.Audio())

# model = inference.Model('/Users/kahrendt/Documents/Hobbies/Machine_Learning/microwakeword/trained_models/hey_mycroft_wide_matchbox_filtered/tflite_stream_state_internal_quant/stream_state_internal_quant.tflite')
model = inference.Model('/Users/kahrendt/Documents/Hobbies/Programming/Git-Repositories/microWakeWord/notebooks/trained_models/hey_mycroft_mixednet_no_initial_conv2/tflite_stream_state_internal_quant/stream_state_internal_quant.tflite')

for clip in audio_dataset:
    null_audio = np.zeros(16*40*20).astype(np.int16)
    # null_audio = np.zeros(16*74*20).astype(np.int16)

    audio = clip["audio"]["array"]
    useful_audio = (audio * 32767).astype(np.int16)

    useful_audio = np.append(null_audio, useful_audio)
    useful_audio = np.append(useful_audio, null_audio)

    probabilities = model.predict_clip(useful_audio)
    # print(clip)
    # print(max(probabilities[50:]))
    if np.max(probabilities[50:]) < 0.5:
        print(np.max(probabilities[50:]))
        print(clip['audio']['path'])

# import wave
# with wave.open('/Volumes/MachineLearning/audio_samples/recorded_samples/hey_mycroft/28.wav') as input_wav:
#     wav_audio =input_wav.readframes(input_wav.getnframes())

# import scipy.io.wavfile as wav

# null_audio = np.zeros(16*20*20).astype(np.int16)
# second_null_audio = np.zeros(16*6*20).astype(np.int16)

# _, wav_audio = wav.read('/Volumes/MachineLearning/audio_samples/recorded_samples/hey_mycroft/personal_samples/31.wav')
# useful_audio = (wav_audio).astype(np.int16)
# useful_audio = np.append(null_audio, useful_audio)
# useful_audio = np.append(useful_audio, second_null_audio)
# probabilities = model.predict_clip(useful_audio)
# import matplotlib.pyplot as plt
# print(np.max(probabilities[100:]))
# plt.plot(probabilities[:])



0.0625
/Volumes/MachineLearning/audio_samples/recorded_samples/hey_mycroft/personal_samples/23.wav
0.375
/Volumes/MachineLearning/audio_samples/recorded_samples/hey_mycroft/personal_samples/32.wav


In [3]:
import openwakeword.utils as utils
import openwakeword.model as model
from tqdm import tqdm
from pathlib import Path

# path_to_clips = [str(i) for i in Path('/Users/kahrendt/Documents/Hobbies/Programming/Git-Repositories/piper-sample-generator/output/mycroft/small').glob('*.wav')]
path_to_clips = [str(i) for i in Path('/Volumes/MachineLearning/audio_samples/recorded_samples/hey_mycroft/personal_samples').glob('*.wav')]

model = model.Model(wakeword_models=['hey mycroft'], inference_framework='onnx')
truncated_clips = clips_handler.clips[:]['audio']
# print(truncated_clips[0]['array'])
# good_counter = 0
bad_counter = 0
good_counter = 0
max_probabilties = []
for clip in tqdm(truncated_clips):
    # print(clip)
    audio_data = clip['array']
    audio_path = clip['path']
    # audio_data = clip["audio"]["array"]
    # spectrogram = clips_handler.generate_augmented_spectrogram(audio_data)
    predictions = model.predict_clip(audio_path)
    test = [prediction['hey mycroft'] for prediction in predictions]
    # print(test)
    max_probabilties.append(max(test))
    if (max(test) < 0.5):
        # print(max(test))
        # print(clip['path'])
        bad_counter +=1
    else:
        good_counter +=1

print(good_counter/len(truncated_clips))

# for clip in path_to_clips:
#     predictions = model.predict_clip(clip)
#     test = [prediction['hey mycroft'] for prediction in predictions]
#     # print(max(test))
#     if (max(test) < 0.5):
#         print(max(test))
#         print(clip)
# utils.download_models()
# utils.bulk_predict(path_to_clips, ['hey mycroft'], inference_framework='onnx')

100%|██████████| 10771/10771 [1:07:31<00:00,  2.66it/s]

0.7622319190418717


In [5]:
import numpy as np
with open('mycroft_real_samples_oww_max_prob.npy', 'wb') as f:
    np.save(f, np.array(max_probabilties))

In [ ]:
import microwakeword.inference as inference
import scipy.io.wavfile as wav
import numpy as np
model = inference.Model('/Users/kahrendt/Documents/Hobbies/Programming/Git-Repositories/microWakeWord/notebooks/trained_models/hey_mycroft_better_rs/tflite_stream_state_internal_quant/stream_state_internal_quant.tflite')
_, wav_audio = wav.read('/Volumes/MachineLearning/audio_samples/recorded_samples/hey_mycroft/7.wav')
useful_audio = (wav_audio).astype(np.int16)
# useful_audio = np.append(null_audio, useful_audio)
with open("audio_raw.bin", "wb") as binary_file:
    binary_file.write(useful_audio.tobytes())
    
# from microwakeword.feature_generation import generate_features_for_clip
# spectrogram, chunk = generate_features_for_clip(useful_audio)



probabilities, chunk = model.predict_clip(useful_audio)
# useful_audio = np.append(useful_audio, null_audio)
print(chunk.shape)
for frame in range(chunk.shape[0]):
    for i in range(0,40):
        print(chunk[frame][i], end=' ')
    print('\n')
# import matplotlib.pyplot as plt
# print(np.max(probabilities[74:]))
# plt.plot(probabilities[:])
